In [5]:
import os
print([i for i in os.listdir() if 'parquet' in i])

['dataframe.parquet']


In [6]:
spark=SparkSession.builder.getOrCreate()
df=spark.read.load('dataframe.parquet',format='parquet')
df.toPandas().head()

,0,1,2,3,4,5,6,7,8,9,...,1994,1995,1996,1997,1998,1999,filename,label,fs,k
0,0.716302,0.740627,0.599815,0.664514,0.786073,0.553410,0.719064,0.703307,0.607023,0.754069,...,0.631046,0.338797,0.639984,0.530173,0.377894,0.593133,extrahls__201101070953.wav,extrahls,367.5,2000
1,0.739262,0.617246,0.803137,0.462727,0.489625,0.699909,0.725328,0.673239,0.572426,0.744569,...,0.664340,0.480305,0.640480,0.608836,0.620766,0.611966,extrahls__201101091153.wav,extrahls,367.5,2000
2,0.913559,0.779044,0.818879,0.500293,0.702190,0.744516,0.643676,0.867989,0.750828,0.856806,...,0.602872,0.318743,0.666170,0.741681,0.656872,0.615473,extrahls__201101152255.wav,extrahls,367.5,2000
3,0.695254,0.564887,0.662952,0.761360,0.728754,0.759764,0.736816,0.707393,0.781271,0.731554,...,0.597531,0.661305,0.667166,0.641461,0.588037,0.717878,extrahls__201101160804.wav,extrahls,367.5,2000
4,0.822854,0.702048,0.398874,0.642867,0.578157,0.544518,0.740318,0.649023,0.726241,0.641135,...,0.572833,0.666673,0.691166,0.456296,0.648299,0.541012,extrahls__201101160808.wav,extrahls,367.5,2000


In [191]:
#train_set - test_set splitting

DUMMY_FRAC=1. 
SEED=1234
DROPLAST=True

train_set,test_set=df.sample(withReplacement=False,fraction=DUMMY_FRAC,seed=SEED).randomSplit([0.75,0.25])

k=int(df.select('k').groupBy().avg().collect()[0]['avg(k)'])
f_cols=[str(i) for i in range(int(k))]
n_train=train_set.select('k').count()
n_test=test_set.select('k').count()

if DROPLAST:
    cats=df.select('label').distinct().count()-1
else:
    cats=df.select('label').distinct().count()
print('train_set: %d datos, test_set: %d datos, categorias: %d'%(n_train,n_test,cats))

train_set: 65 datos, test_set: 19 datos, categorias: 2


In [192]:
import numpy as np
import h5py as h5
from keras.models import Sequential
from keras.layers import Conv1D,MaxPooling1D, Dropout,Dense,Activation,Flatten
from keras.utils import to_categorical

In [193]:
from pyspark.ml.feature import VectorAssembler,StringIndexer,OneHotEncoder,PCA
from pyspark.ml import Pipeline

pca_k=140

#input data preprocessing
vecAssembler = VectorAssembler(inputCols=f_cols, outputCol="freq_channels_")
vecAssembler0 = VectorAssembler(inputCols=f_cols, outputCol="freq_channels")
pca=PCA(k=pca_k,inputCol='freq_channels_',outputCol='freq_channels')

#output data preprocessing
Indexer = StringIndexer(inputCol='label',outputCol='labelIndex')
OneHot= OneHotEncoder(inputCol='labelIndex',outputCol='out',dropLast=DROPLAST)


In [194]:
import numpy as np
import pandas as pd
from keras.optimizers import Adagrad
from keras.regularizers import l2

if pca_k>0:
    x_PreProc=Pipeline(stages=[vecAssembler,pca])
else:
    x_PreProc=Pipeline(stages=[vecAssembler0])
    pca_k=k

x_PreProc=x_PreProc.fit(train_set)

y_PreProc=Pipeline(stages=[Indexer,OneHot])
y_PreProc=y_PreProc.fit(train_set)

x_train=np.array(x_PreProc.transform(train_set).select('freq_channels').collect())
x_train=x_train.reshape((n_train,pca_k,1))

y_train=np.array(y_PreProc.transform(train_set).select('out').collect(),dtype='int32')
y_train=y_train.reshape((n_train,cats))

x_test=np.array(x_PreProc.transform(test_set).select('freq_channels').collect())
x_test=x_test.reshape((n_test,pca_k,1))

y_test=np.array(y_PreProc.transform(test_set).select('out').collect(),dtype='int32')
y_test=y_test.reshape(n_test,cats)
print('pca_k : %d'%pca_k)

pca_k : 140


In [205]:
'''
filters 199, batch_size 1, kernel_size 16, n_hidden 23, pool_size: 18
accuracy 0.6154, recall 0.8750 , 0.4000
'''

from sklearn.metrics import recall_score
vec_round=np.vectorize(lambda x: int(round(x)))

RANDOM_SEARCH=0
filters=199
kernel_size=16
pool_size=18
batch_size=1
rate=0.7
activation='softmax'
activation_l='softmax'
n_hidden=23
l2_r=0.
l2_cnn=0.
l2_r2=.001

accuracy=0.0
best_accuracy=0
best_params=None

epochs= 40 if RANDOM_SEARCH else 240
acc_target = 0.88 if RANDOM_SEARCH else 0.01


while (accuracy < acc_target): 
    
    if RANDOM_SEARCH:
        filters=int(np.random.uniform(30,120))
        kernel_size=int(np.random.uniform(2,120))
        n_hidden=int(np.random.uniform(5,80))
        pool_size=int(np.random.uniform(1,50))
        batch_size=int(np.random.uniform(1,15))
    
    model=Sequential()
    
    model.add(Conv1D(filters=filters,kernel_size=kernel_size,padding='causal',activation=activation_l,kernel_regularizer=l2(l2_cnn),input_shape=(pca_k,1)))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Dropout(rate=rate))
    
    model.add(Flatten())
    
    model.add(Dense(n_hidden,kernel_regularizer=l2(l2_r)))
    model.add(Activation(activation))

    model.add(Dense(cats,kernel_regularizer=l2(l2_r2)))
    model.add(Activation(activation))
    
    model.compile(loss='categorical_crossentropy',optimizer=Adagrad(),metrics=['accuracy'])
    model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=(not RANDOM_SEARCH),validation_data=(x_test,y_test))
            
    evaluation=model.evaluate(x_test,y_test)
    evaltrain=model.evaluate(x_train,y_train)
    yt_pred=vec_round(model.predict(x_test))
    recall1,recall2=recall_score(y_test[:,0],yt_pred[:,0]),recall_score(y_test[:,1],yt_pred[:,1])
    
    accuracy=evaluation[1]
    if accuracy>=best_accuracy:
        best_accuracy=accuracy
        best_params={'filters': filters,'batch_size': batch_size,'kernel_size':kernel_size,'n_hidden':n_hidden,'pool_size':pool_size}
        print('\n----------------------------------------------------------------------')
        print('filters %d, batch_size %d, kernel_size %d, n_hidden %d, pool_size: %d'%(best_params['filters'],best_params['batch_size'],best_params['kernel_size'],best_params['n_hidden'],best_params['pool_size']))
        print('accuracy %1.4f, recall %1.4f , %1.4f'%(best_accuracy,recall1,recall2))
        print('----------------------------------------------------------------------\n')
    model.summary()
    print('TRAIN: loss= %1.5f, accuracy= %1.5f |TEST: loss= %1.5f,accuracy= %1.5f)'%(evaltrain[0],evaltrain[1],evaluation[0],evaluation[1]))

print('******************************')
print(best_params)
print(best_accuracy)

Train on 65 samples, validate on 19 samples
Epoch 1/240
65/65 [==============================] - 12s - loss: 0.5422 - acc: 0.4000 - val_loss: 0.5446 - val_acc: 0.5263
Epoch 2/240
65/65 [==============================] - 0s - loss: 0.5386 - acc: 0.3538 - val_loss: 0.5472 - val_acc: 0.5789
Epoch 3/240
65/65 [==============================] - 0s - loss: 0.5375 - acc: 0.4769 - val_loss: 0.5492 - val_acc: 0.6842
Epoch 4/240
65/65 [==============================] - 0s - loss: 0.5368 - acc: 0.4923 - val_loss: 0.5503 - val_acc: 0.4737
Epoch 5/240
65/65 [==============================] - 0s - loss: 0.5354 - acc: 0.6462 - val_loss: 0.5513 - val_acc: 0.4737
Epoch 6/240
65/65 [==============================] - 0s - loss: 0.5350 - acc: 0.6000 - val_loss: 0.5525 - val_acc: 0.4737
Epoch 7/240
65/65 [==============================] - 0s - loss: 0.5357 - acc: 0.6000 - val_loss: 0.5531 - val_acc: 0.4737
Epoch 8/240
65/65 [==============================] - 0s - loss: 0.5340 - acc: 0.6308 - val_loss: 0.55

65/65 [==============================] - 0s - loss: 0.5087 - acc: 0.6615 - val_loss: 0.4676 - val_acc: 0.9474
Epoch 134/240
65/65 [==============================] - 0s - loss: 0.4992 - acc: 0.7231 - val_loss: 0.4673 - val_acc: 0.9474
Epoch 135/240
65/65 [==============================] - 0s - loss: 0.4843 - acc: 0.7231 - val_loss: 0.4684 - val_acc: 0.9474
Epoch 136/240
65/65 [==============================] - 0s - loss: 0.4679 - acc: 0.7231 - val_loss: 0.4693 - val_acc: 0.9474
Epoch 137/240
65/65 [==============================] - 0s - loss: 0.4925 - acc: 0.6769 - val_loss: 0.4678 - val_acc: 0.9474
Epoch 138/240
65/65 [==============================] - 0s - loss: 0.4707 - acc: 0.8308 - val_loss: 0.4694 - val_acc: 0.9474
Epoch 139/240
65/65 [==============================] - 0s - loss: 0.5147 - acc: 0.6769 - val_loss: 0.4681 - val_acc: 0.9474
Epoch 140/240
65/65 [==============================] - 0s - loss: 0.4970 - acc: 0.6308 - val_loss: 0.4693 - val_acc: 0.9474
Epoch 141/240
65/65 [=

In [206]:
from bokeh.plotting import figure,show
from bokeh.io import output_notebook
from bokeh.layouts import row
output_notebook()
xp=x_test
yp=y_test

y_pred=model.predict(xp)

p=figure()
p.scatter(x=yp[:,0],y=y_pred[:,0],color='blue')
q=figure()
q.scatter(x=yp[:,1],y=y_pred[:,1],color='red')
if DROPLAST:
    show(row([p,q]))
else:
    r=figure()
    r.scatter(x=yp[:,2],y=y_pred[:,2],color='green')
    show(row([p,q,r]))


Loading BokehJS ...

In [220]:
from sklearn.metrics import confusion_matrix

confusion=[]
th=[0.5,0.5,0.5]
def vec_round(x,th):
    v=np.vectorize(lambda x: 1 if x>=th else 0)
    return v(x)

for cat in [0,1]:    
    y_predBin=vec_round(y_pred[:,cat].flatten(),th[cat])
    cm=confusion_matrix(yp[:,cat].flatten(),y_predBin)
    cm=cm/sum(cm.flatten())
    confusion.append(cm)
    print('confussion matrix for output %d'%(cat+1))
    print(cm)
    print('recall: %1.4f'%(1-cm[0,1]/(cm[0,1]+cm[1,1])))
    print('precision: %1.4f'%(1-cm[1,0]/(cm[1,0]+cm[1,1])))

[[ 0.47368421  0.26315789]
 [ 0.          0.26315789]]
recall: 0.5000
precision: 1.0000
[[ 0.47368421  0.        ]
 [ 0.05263158  0.47368421]]
recall: 1.0000
precision: 0.9000


In [221]:
from sklearn.metrics import precision_recall_curve
p=figure(x_range=(0,1),y_range=(0,1))
precision0, recall0, thresholds0 = precision_recall_curve(yp[:,0],y_pred[:,0])
precision1, recall1, thresholds1 = precision_recall_curve(yp[:,1],y_pred[:,1])
#precision2, recall2, thresholds2 = precision_recall_curve(yp[:,2],y_pred[:,2])
p.line(x=recall0,y=precision0,color='blue')
p.line(x=recall1,y=precision1,color='red')
#p.line(x=recall2,y=precision2,color='green')
show(p)


In [218]:
import h5py
filename='model(bs%d f%d ks%d h%d ps%d) acc%1.2f'%(best_params['batch_size'], \
                                                   best_params['filters'], \
                                                   best_params['kernel_size'], \
                                                   best_params['n_hidden'], \
                                                   best_params['pool_size'], \
                                                   best_accuracy)
json_str=model.to_json()
with open(filename+'_.json','w') as f:
    f.write(json_str)

model.save_weights(filename+'_.h5')


print([i for i in os.listdir() if 'model(' in i])

['model(bs1 f199 ks16 h23 ps18) acc0.95_.json', 'model(bs1 f199 ks16 h23 ps18) acc0.95_.h5']


In [215]:
[os.remove(i) for i in os.listdir() if 'model(' in i]

[None, None]

In [216]:
best_params

{'batch_size': 1,
 'filters': 199,
 'kernel_size': 16,
 'n_hidden': 23,
 'pool_size': 18}